In [1]:
#one level up into project folder
import os
os.chdir("..")

#print("Current working directory:", os.getcwd())

In [2]:
import os
import sys
import glob
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

import mlflow
import dagshub

from visioninfantnet.utils.ml_utils.metric.classification_metric import (
    get_classification_score,
)
from visioninfantnet.exception.exception import VisionInfantNetException

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
print(f"Using device: {device}")

Using device: cuda


In [3]:
from dotenv import load_dotenv
load_dotenv()
mlflow_uri = os.getenv("MLFLOW_TRACKING_URI")
mlflow.set_tracking_uri(mlflow_uri)
dagshub.init(repo_owner='arunps12', repo_name='VisionInfantNet', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)


Accessing as arunps12

Initialized MLflow to track repo "arunps12/VisionInfantNet"

Repository arunps12/VisionInfantNet initialized!

🏃 View run bright-bee-313 at: https://dagshub.com/arunps12/VisionInfantNet.mlflow/#/experiments/0/runs/5402700a169444218642667073e5f9a4
🧪 View experiment at: https://dagshub.com/arunps12/VisionInfantNet.mlflow/#/experiments/0


In [5]:
#Paths and data loading helper

ARTIFACT_ROOT = os.getenv("ARTIFACT_ROOT")

TRAIN_IMG_DIR = os.path.join(ARTIFACT_ROOT, "data_transformation", "spectrograms", "train")
VAL_IMG_DIR   = os.path.join(ARTIFACT_ROOT, "data_transformation", "spectrograms", "valid")

TRAIN_LABEL_NPY = os.path.join(ARTIFACT_ROOT, "data_transformation", "features", "train_labels.npy")
VAL_LABEL_NPY   = os.path.join(ARTIFACT_ROOT, "data_transformation", "features", "valid_labels.npy")

print(TRAIN_IMG_DIR)
print(VAL_IMG_DIR)
print(TRAIN_LABEL_NPY)
print(VAL_LABEL_NPY)


/itf-fi-ml/home/arunps/Projects/VisionInfantNet/artifacts/12_02_2025_10_27_46/data_transformation/spectrograms/train
/itf-fi-ml/home/arunps/Projects/VisionInfantNet/artifacts/12_02_2025_10_27_46/data_transformation/spectrograms/valid
/itf-fi-ml/home/arunps/Projects/VisionInfantNet/artifacts/12_02_2025_10_27_46/data_transformation/features/train_labels.npy
/itf-fi-ml/home/arunps/Projects/VisionInfantNet/artifacts/12_02_2025_10_27_46/data_transformation/features/valid_labels.npy


In [6]:
# Load labels and image paths
train_labels = np.load(TRAIN_LABEL_NPY)
val_labels   = np.load(VAL_LABEL_NPY)

train_image_paths = sorted(glob.glob(os.path.join(TRAIN_IMG_DIR, "*.png")))
val_image_paths   = sorted(glob.glob(os.path.join(VAL_IMG_DIR, "*.png")))

print(len(train_image_paths), len(train_labels))
print(len(val_image_paths), len(val_labels))

num_classes = len(np.unique(train_labels))
num_classes


3600 3600
3580 3580


5

In [7]:
import numpy as np

unique_labels = sorted(set(train_labels) | set(val_labels))
label_to_idx = {lab: i for i, lab in enumerate(unique_labels)}
idx_to_label = {i: lab for lab, i in label_to_idx.items()}

print("Label mapping:", label_to_idx)
num_classes = len(unique_labels)
print("num_classes:", num_classes)


Label mapping: {np.str_('Canonical'): 0, np.str_('Crying'): 1, np.str_('Junk'): 2, np.str_('Laughing'): 3, np.str_('Non-canonical'): 4}
num_classes: 5


In [8]:
train_labels_idx = np.array([label_to_idx[l] for l in train_labels])
val_labels_idx   = np.array([label_to_idx[l] for l in val_labels])
print(train_labels_idx[:10])
print(val_labels_idx[:10])

[1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]


In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms as T
from PIL import Image

class SpectrogramImageDataset(Dataset):
    def __init__(self, image_paths, labels, train: bool = True):
        self.image_paths = image_paths
        self.labels = labels

       
        base_transforms = [
            T.Resize((224, 224)),
            T.ToTensor(),              # -> [1, H, W], values [0,1]
        ]

        self.transforms = T.Compose(base_transforms)

        # ---- Model normalization ([-1,1]) ----
        self.normalize = T.Normalize(
            mean=[0.5, 0.5, 0.5],
            std=[0.5, 0.5, 0.5],
        )

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        label = self.labels[idx]

        # Load grayscale PNG (0..255)
        img = Image.open(path).convert("L")

        # Apply transforms → [1,224,224]
        img = self.transforms(img)

        # Convert grayscale -> RGB by repeating channels
        img = img.repeat(3, 1, 1)  # [3,224,224]

        # Normalize to [-1,1]
        img = self.normalize(img)

        return img, torch.tensor(label, dtype=torch.long)


In [10]:
BATCH_SIZE = 64  # override per experiment

train_dataset = SpectrogramImageDataset(train_image_paths, train_labels_idx, train=True)
val_dataset   = SpectrogramImageDataset(val_image_paths, val_labels_idx, train=False)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)


In [11]:
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # downsample by /2
        )

    def forward(self, x):
        return self.block(x)


class BaseCNNSpectrogram(nn.Module):
    def __init__(
        self,
        num_classes: int,
        base_channels: int = 32,
        num_blocks: int = 3,
    ):
        super().__init__()

        channels = [3] + [base_channels * (2 ** i) for i in range(num_blocks)]
        conv_blocks = []
        for i in range(num_blocks):
            conv_blocks.append(ConvBlock(channels[i], channels[i+1]))
        self.conv = nn.Sequential(*conv_blocks)

        last_channels = channels[-1]
        spatial_size = 224 // (2 ** num_blocks)  # after MaxPool(2) num_blocks times
        self.flatten_dim = last_channels * spatial_size * spatial_size

        # Single DNN classification layer: Flatten -> Linear -> logits
        self.fc = nn.Linear(self.flatten_dim, num_classes)

    def forward(self, x):
        x = self.conv(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [17]:
import torch.nn as nn
import torch.nn.init as init

def init_cnn_weights(m, init_method: str):
    """
    Apply a chosen initialization method to Conv and Linear layers.
    """
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        if init_method == "kaiming_normal":
            init.kaiming_normal_(m.weight, nonlinearity="relu")
        elif init_method == "kaiming_uniform":
            init.kaiming_uniform_(m.weight, nonlinearity="relu")
        elif init_method == "xavier_normal":
            init.xavier_normal_(m.weight)
        elif init_method == "xavier_uniform":
            init.xavier_uniform_(m.weight)
        elif init_method == "orthogonal":
            init.orthogonal_(m.weight)
        elif init_method == "pytorch_default":
            pass
        else:
            raise ValueError(f"Unknown init_method: {init_method}")

        if m.bias is not None:
            nn.init.zeros_(m.bias)


In [18]:
import torch.nn.functional as F
from torch import optim
import mlflow.pytorch
def train_one_cnn_experiment(
    num_blocks: int,
    base_channels: int,
    train_dataset,
    val_dataset,
    num_classes: int,
    batch_size: int = 64,
    lr: float = 1e-3,
    weight_decay: float = 1e-4,
    epochs: int = 15,
    run_name: str = None,
    MODEL_DIR: str = "saved_models",
    init_method: str = "pytorch_default",
):
    # Dataloaders for training and validation # update loaders if batch_size changes
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
    )

    model = BaseCNNSpectrogram(
        num_classes=num_classes,
        base_channels=base_channels,
        num_blocks=num_blocks,
    ).to(device)

    # Initialize weights
    model.apply(lambda m: init_cnn_weights(m, init_method))

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay,
    )

    if run_name is None:
        run_name = f"cnn_blocks{num_blocks}_base{base_channels}_{init_method}"

    best_val_uar = 0.0
    best_epoch = -1

    with mlflow.start_run(run_name=run_name):
        # Log hyperparameters
        mlflow.log_param("num_blocks", num_blocks)
        mlflow.log_param("base_channels", base_channels)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("lr", lr)
        mlflow.log_param("weight_decay", weight_decay)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("model_type", "BaseCNN_flatten_linear")
        mlflow.log_param("init_method", init_method)

        for epoch in range(epochs):
            # ---- train ----
            model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0

            for imgs, labels in train_loader:
                imgs = imgs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * imgs.size(0)
                preds = outputs.argmax(dim=1)
                train_correct += (preds == labels).sum().item()
                train_total += labels.size(0)

            train_loss /= train_total
            train_acc = train_correct / train_total

            # ---- validation ----
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0
            all_true, all_pred = [], []

            with torch.no_grad():
                for imgs, labels in val_loader:
                    imgs = imgs.to(device)
                    labels = labels.to(device)

                    outputs = model(imgs)
                    loss = criterion(outputs, labels)

                    val_loss += loss.item() * imgs.size(0)
                    preds = outputs.argmax(dim=1)

                    val_correct += (preds == labels).sum().item()
                    val_total += labels.size(0)

                    all_true.extend(labels.cpu().numpy().tolist())
                    all_pred.extend(preds.cpu().numpy().tolist())

            val_loss /= val_total
            val_acc = val_correct / val_total

            # classification metrics
            metrics = get_classification_score(
                y_true=all_true,
                y_pred=all_pred,
                average="weighted",
            )

            # Log metrics to MLflow
            mlflow.log_metric("train_loss", train_loss, step=epoch)
            mlflow.log_metric("train_acc", train_acc, step=epoch)
            mlflow.log_metric("val_loss", val_loss, step=epoch)
            mlflow.log_metric("val_acc", val_acc, step=epoch)
            mlflow.log_metric("val_f1", metrics.f1_score, step=epoch)
            mlflow.log_metric("val_precision", metrics.precision_score, step=epoch)
            mlflow.log_metric("val_recall", metrics.recall_score, step=epoch)
            mlflow.log_metric("val_uar", metrics.uar, step=epoch)

            if metrics.uar > best_val_uar:
                best_val_uar = metrics.uar
                best_epoch = epoch
                os.makedirs(MODEL_DIR, exist_ok=True)
                model_path = os.path.join(
                                            MODEL_DIR,
                                                f"best_model_blocks{num_blocks}_base{base_channels}_init_{init_method}.pt"
                                            )
                torch.save(model.state_dict(), model_path)
                mlflow.log_artifact(model_path)
                #mlflow.pytorch.log_model(model, artifact_path="pytorch_model")
            print(
                f"[{run_name}] Epoch {epoch+1}/{epochs} | "
                f"Train Loss: {train_loss:.4f} Acc: {train_acc:.3f} | "
                f"Val Loss: {val_loss:.4f} Acc: {val_acc:.3f} | "
                f"F1: {metrics.f1_score:.3f} UAR: {metrics.uar:.3f}"
            )

        mlflow.log_metric("best_val_uar", best_val_uar)
        mlflow.log_param("best_epoch", best_epoch)

    return model, best_val_uar


In [25]:
# ---- MLflow Experiment 1: base_channels sweep ----
mlflow.set_experiment("cnn_exp1_base_channels")

BASE_CHANNEL_LIST = [8, 16, 32, 64]
FIXED_NUM_BLOCKS = 3

base_channel_results = {}

for bc in BASE_CHANNEL_LIST:
    print(f"\n=== Running base_channels={bc}, num_blocks={FIXED_NUM_BLOCKS} ===")
    run_name = f"base{bc}_blocks{FIXED_NUM_BLOCKS}"

    model, best_uar = train_one_cnn_experiment(
        num_blocks=FIXED_NUM_BLOCKS,
        base_channels=bc,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        num_classes=num_classes,
        batch_size=64,
        lr=1e-3,
        weight_decay=1e-4,
        epochs=15,
        run_name=run_name,
    )

    base_channel_results[bc] = best_uar
    print(f"--> base_channels={bc}: best UAR={best_uar:.4f}")



=== Running base_channels=8, num_blocks=3 ===
[base8_blocks3] Epoch 1/15 | Train Loss: 3.2883 Acc: 0.355 | Val Loss: 1.3928 Acc: 0.456 | F1: 0.304 UAR: 0.201
[base8_blocks3] Epoch 2/15 | Train Loss: 1.2974 Acc: 0.420 | Val Loss: 1.5288 Acc: 0.378 | F1: 0.224 UAR: 0.204
[base8_blocks3] Epoch 3/15 | Train Loss: 1.2679 Acc: 0.443 | Val Loss: 1.2628 Acc: 0.423 | F1: 0.382 UAR: 0.199
[base8_blocks3] Epoch 4/15 | Train Loss: 1.1820 Acc: 0.470 | Val Loss: 1.7377 Acc: 0.254 | F1: 0.206 UAR: 0.239
[base8_blocks3] Epoch 5/15 | Train Loss: 1.1249 Acc: 0.500 | Val Loss: 1.5035 Acc: 0.378 | F1: 0.234 UAR: 0.206
[base8_blocks3] Epoch 6/15 | Train Loss: 1.0694 Acc: 0.518 | Val Loss: 1.3580 Acc: 0.404 | F1: 0.347 UAR: 0.203
[base8_blocks3] Epoch 7/15 | Train Loss: 1.0331 Acc: 0.541 | Val Loss: 1.3205 Acc: 0.430 | F1: 0.371 UAR: 0.217
[base8_blocks3] Epoch 8/15 | Train Loss: 0.9006 Acc: 0.604 | Val Loss: 1.3381 Acc: 0.396 | F1: 0.376 UAR: 0.218
[base8_blocks3] Epoch 9/15 | Train Loss: 0.9031 Acc: 0.60

In [15]:
# ---- MLflow Experiment 2: num_blocks sweep ----
mlflow.set_experiment("cnn_exp2_num_blocks")

best_base_channels = 64 # best from exp1 
NUM_BLOCKS_LIST = [2, 3, 4, 5, 6]

blocks_results = {}

for nb in NUM_BLOCKS_LIST:
    print(f"\n=== Running num_blocks={nb}, base_channels={best_base_channels} ===")
    run_name = f"blocks{nb}_base{best_base_channels}"

    model, best_uar = train_one_cnn_experiment(
        num_blocks=nb,
        base_channels=best_base_channels,   
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        num_classes=num_classes,
        batch_size=64,
        lr=1e-3,
        weight_decay=1e-4,
        epochs=15,
        run_name=run_name,
    )

    blocks_results[nb] = best_uar
    print(f"--> num_blocks={nb}: best UAR={best_uar:.4f}")

# pick best num_blocks
best_num_blocks = max(blocks_results, key=blocks_results.get)
best_blocks_uar = blocks_results[best_num_blocks]
print(f"\n*** Best num_blocks = {best_num_blocks} with UAR={best_blocks_uar:.4f} ***")



=== Running num_blocks=2, base_channels=64 ===
[blocks2_base64] Epoch 1/15 | Train Loss: 44.5425 Acc: 0.347 | Val Loss: 13.9489 Acc: 0.377 | F1: 0.235 UAR: 0.200
[blocks2_base64] Epoch 2/15 | Train Loss: 15.4631 Acc: 0.367 | Val Loss: 13.3996 Acc: 0.429 | F1: 0.375 UAR: 0.206
[blocks2_base64] Epoch 3/15 | Train Loss: 15.0422 Acc: 0.389 | Val Loss: 16.0260 Acc: 0.282 | F1: 0.211 UAR: 0.217
[blocks2_base64] Epoch 4/15 | Train Loss: 8.9733 Acc: 0.429 | Val Loss: 16.4752 Acc: 0.357 | F1: 0.265 UAR: 0.206
[blocks2_base64] Epoch 5/15 | Train Loss: 9.8473 Acc: 0.441 | Val Loss: 9.6741 Acc: 0.432 | F1: 0.398 UAR: 0.210
[blocks2_base64] Epoch 6/15 | Train Loss: 5.7654 Acc: 0.493 | Val Loss: 6.7503 Acc: 0.435 | F1: 0.378 UAR: 0.207
[blocks2_base64] Epoch 7/15 | Train Loss: 4.3215 Acc: 0.536 | Val Loss: 12.2780 Acc: 0.315 | F1: 0.274 UAR: 0.215
[blocks2_base64] Epoch 8/15 | Train Loss: 4.7584 Acc: 0.506 | Val Loss: 6.6528 Acc: 0.413 | F1: 0.387 UAR: 0.206
[blocks2_base64] Epoch 9/15 | Train Loss

In [ ]:
# ---- MLflow Experiment 3: initialization sweep ----
mlflow.set_experiment("cnn_exp3_init_methods")

INIT_METHODS = [
    "pytorch_default",
    "kaiming_normal",
    "kaiming_uniform",
    "xavier_normal",
    "xavier_uniform",
    "orthogonal",
]

init_results = {}

for init_method in INIT_METHODS:
    print(f"\n=== Running init_method={init_method}, num_blocks=5, base_channels=64 ===")
    run_name = f"init_{init_method}_blocks5_base64"

    model, best_uar = train_one_cnn_experiment(
        num_blocks=5,                  # from previous best
        base_channels=64,              # from previous best
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        num_classes=num_classes,
        batch_size=64,
        lr=1e-3,
        weight_decay=1e-4,
        epochs=15,
        run_name=run_name,
        init_method=init_method,
    )

    init_results[init_method] = best_uar
    print(f"--> init_method={init_method}: best UAR={best_uar:.4f}")

best_init = max(init_results, key=init_results.get)
print("\n*** Initialization summary ***")
for k, v in init_results.items():
    print(f"{k:16s} -> UAR={v:.4f}")

print(f"\n>>> Best init_method = {best_init} with UAR={init_results[best_init]:.4f}")


2025/12/02 15:26:23 INFO mlflow.tracking.fluent: Experiment with name 'cnn_exp3_init_methods' does not exist. Creating a new experiment.



=== Running init_method=pytorch_default, num_blocks=5, base_channels=64 ===
[init_pytorch_default_blocks5_base64] Epoch 1/15 | Train Loss: 5.6753 Acc: 0.346 | Val Loss: 2.2290 Acc: 0.255 | F1: 0.281 UAR: 0.217
[init_pytorch_default_blocks5_base64] Epoch 2/15 | Train Loss: 2.0815 Acc: 0.359 | Val Loss: 2.1406 Acc: 0.280 | F1: 0.296 UAR: 0.223
[init_pytorch_default_blocks5_base64] Epoch 3/15 | Train Loss: 1.8716 Acc: 0.368 | Val Loss: 1.4183 Acc: 0.338 | F1: 0.353 UAR: 0.234
[init_pytorch_default_blocks5_base64] Epoch 4/15 | Train Loss: 1.4672 Acc: 0.389 | Val Loss: 1.4271 Acc: 0.458 | F1: 0.292 UAR: 0.202
[init_pytorch_default_blocks5_base64] Epoch 5/15 | Train Loss: 1.3393 Acc: 0.407 | Val Loss: 1.2845 Acc: 0.455 | F1: 0.323 UAR: 0.201
[init_pytorch_default_blocks5_base64] Epoch 6/15 | Train Loss: 1.2971 Acc: 0.412 | Val Loss: 1.3094 Acc: 0.359 | F1: 0.354 UAR: 0.227
[init_pytorch_default_blocks5_base64] Epoch 7/15 | Train Loss: 1.3232 Acc: 0.423 | Val Loss: 1.3830 Acc: 0.458 | F1: 0.

: 